In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import glob
import os

import warnings
warnings.filterwarnings(action='ignore')

In [28]:
file = pd.read_csv('raw/서울특별시 병원 인허가 정보.csv', encoding='cp949')
file

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,진료과목내용명,지정취소일자,완화의료지정형태,완화의료담당부서명,구급차특수,구급차일반,총인원,구조사수,허가병상수,최초지정일자
0,3000000,PHMA220083000034021200001,20080624,NaN,3,폐업,3,폐업,20181001.0,NaN,...,한방부인과 침구과 사상체질과 한방재활의학과 한방신경정신과 한방안?이비인후?피부과 한...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
1,3000000,PHMA220103000034021200001,20080912,NaN,3,폐업,3,폐업,20111007.0,NaN,...,피부과 소아청소년과 이비인후과 외과 재활의학과 내과,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
2,3000000,PHMA219823000034021200001,19820924,NaN,3,폐업,3,폐업,20130108.0,NaN,...,영상의학과 내과,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
3,3000000,PHMA220063000034021200001,20060803,NaN,3,폐업,3,폐업,20120217.0,NaN,...,예방치과 구강병리과 구강악안면방사선과 구강내과 치과보존과 치주과 소아치과 치과교정과...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
4,3000000,PHMA219663000034021200001,19660119,NaN,3,폐업,3,폐업,20080930.0,NaN,...,치과 신경과 응급의학과 영상의학과 흉부외과 재활의학과 가정의학과 진단검사의학과 병리...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,3200000,PHMA219843200033021200001,19800620,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
808,3180000,PHMA219933180034021200001,19820419,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
809,3180000,PHMA219853180034021200001,19800111,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
810,3240000,PHMA220203240033021200001,20200812,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1. 필요한 컬럼만 추출

In [29]:
df = file[['인허가일자', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '지번주소', '도로명주소', '사업장명', '최종수정일자']]
df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,최종수정일자
0,20080624,폐업,20181001.0,NaN,NaN,서울특별시 종로구 돈의동 78번지,서울특별시 종로구 돈화문로9길 26 (돈의동),춘원당한방병원,20181001134047
1,20080912,폐업,20111007.0,NaN,NaN,서울특별시 종로구 창신1동 327번지 2호 금호팰리스 4층,서울특별시 종로구 지봉로 29 (창신동),동대문요양병원,20111030144152
2,19820924,폐업,20130108.0,NaN,NaN,서울특별시 종로구 관훈동 196-10,NaN,해정병원,20130108143140
3,20060803,폐업,20120217.0,NaN,NaN,서울특별시 종로구 종로2가 75번지 9호,서울특별시 종로구 종로 78 (종로2가),밀레니엄치과병원,20120217111529
4,19660119,폐업,20080930.0,NaN,NaN,서울특별시 종로구 종로6가 70,NaN,이대부속동대문병원,20091019105921
...,...,...,...,...,...,...,...,...,...
807,19800620,영업중,NaN,NaN,NaN,서울특별시 관악구 신림동 10번지 715호 양지병원,서울특별시 관악구 남부순환로 1636 양지병원 (신림동),에이치플러스 양지병원,20200904110711
808,19820419,영업중,NaN,NaN,NaN,서울특별시 영등포구 영등포동7가 94번지 200호,서울특별시 영등포구 버드나루로7길 12 (영등포동7가 한강성심병원),한림대학교 한강성심병원,20200924205150
809,19800111,영업중,NaN,NaN,NaN,서울특별시 영등포구 대림동 948번지 1호,서울특별시 영등포구 신길로 1 (대림동 강남성심병원),한림대학교 강남성심병원,20200924204630
810,20200812,영업중,NaN,NaN,NaN,서울특별시 강동구 길동 420,서울특별시 강동구 진황도로 124 (길동),서울성심한방병원,20200903141153


### 2. NaN값 처리
#### 1) 폐업일자

In [84]:
df.dtypes

인허가일자        int64
상세영업상태명     object
폐업일자       float64
휴업시작일자     float64
휴업종료일자     float64
지번주소        object
도로명주소       object
사업장명        object
최종수정일자       int64
dtype: object

In [30]:
# 컬럼 타입 변환 : 폐업일자에 맞추기
df['최종수정일자'] = df['최종수정일자'].astype(float)

In [31]:
# df 분리
## open_null_df : 영업중이고 폐업일자가 null인 병원
## rest_null_df : 휴업/전출이고 폐업일자가 null인 병원
## closed_null_df : 영업 종료했고 폐업일자가 null인 병원
## notnull_df : 폐업일자가 null이 아닌 병원

open_null_df = df[(df['상세영업상태명']=='영업중')&(df['폐업일자'].isna())].reset_index(drop=True)
rest_null_df = df[(df['상세영업상태명']=='휴업')|(df['상세영업상태명']=='전출')&(df['폐업일자'].isna())].reset_index(drop=True)
closed_null_df = df[(df['상세영업상태명']!='영업중')&(df['상세영업상태명']!='휴업')&(df['상세영업상태명']!='전출')&(df['폐업일자'].isna())].reset_index(drop=True)
notnull_df = df[df['폐업일자'].notnull()].reset_index(drop=True)
notnull_df['최종수정일자'] = notnull_df['최종수정일자'].astype('int64')
print(len(open_null_df), len(rest_null_df), len(closed_null_df), len(notnull_df))

529 8 33 242


In [32]:
# 영업중이고 폐업일자가 null인 병원의 폐업일자, 휴업시작일자, 휴업종료일자를 0.0으로 채우기
open_null_df['폐업일자'].fillna(0.0, inplace=True)
open_null_df['휴업시작일자'].fillna(0.0, inplace=True)
open_null_df['휴업종료일자'].fillna(0.0, inplace=True)
open_null_df['최종수정일자'] = open_null_df['최종수정일자'].astype('int64')
open_null_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,최종수정일자
0,19891012,영업중,0.0,0.0,0.0,NaN,서울특별시 종로구 삼청로10길 13 (삼청동),서울지구병원,20200228160711
1,20040901,영업중,0.0,0.0,0.0,NaN,서울특별시 종로구 대학로 101 (연건동),서울대학교치과병원,20190925074803
2,20190124,영업중,0.0,0.0,0.0,서울특별시 강동구 성내동 452번지 느루요양병원빌딩,서울특별시 강동구 강동대로 143-52 느루요양병원빌딩 (성내동),느루요양병원,20200708095851
3,19870424,영업중,0.0,0.0,0.0,서울특별시 종로구 무악동 32번지 2호,서울특별시 종로구 통일로 256 (무악동),세란병원,20200709095440
4,20200708,영업중,0.0,0.0,0.0,서울특별시 강서구 마곡동 799-11 파인스퀘어,서울특별시 강서구 마곡중앙4로 22 파인스퀘어 34층 (마곡동),보금한방병원,20200810180942
...,...,...,...,...,...,...,...,...,...
524,19800620,영업중,0.0,0.0,0.0,서울특별시 관악구 신림동 10번지 715호 양지병원,서울특별시 관악구 남부순환로 1636 양지병원 (신림동),에이치플러스 양지병원,20200904110711
525,19820419,영업중,0.0,0.0,0.0,서울특별시 영등포구 영등포동7가 94번지 200호,서울특별시 영등포구 버드나루로7길 12 (영등포동7가 한강성심병원),한림대학교 한강성심병원,20200924205150
526,19800111,영업중,0.0,0.0,0.0,서울특별시 영등포구 대림동 948번지 1호,서울특별시 영등포구 신길로 1 (대림동 강남성심병원),한림대학교 강남성심병원,20200924204630
527,20200812,영업중,0.0,0.0,0.0,서울특별시 강동구 길동 420,서울특별시 강동구 진황도로 124 (길동),서울성심한방병원,20200903141153


In [33]:
# 영업중이고 폐업일자가 null인 병원의 폐업일자, 휴업시작일자, 휴업종료일자를 0.0으로 채우기
rest_null_df['폐업일자'].fillna(0.0, inplace=True)
rest_null_df['휴업시작일자'].fillna(0.0, inplace=True)
rest_null_df['휴업종료일자'].fillna(0.0, inplace=True)
rest_null_df['최종수정일자'] = open_null_df['최종수정일자'].astype('int64')
rest_null_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,최종수정일자
0,20110602,휴업,0.0,20191120.0,20201119.0,서울특별시 중구 을지로3가 302번지 1호,서울특별시 중구 을지로14길 12 (을지로3가),을지치과병원,20200228160711
1,20170106,휴업,0.0,20200801.0,20201231.0,NaN,서울특별시 성동구 왕십리로 382 (하왕십리동),서울동인병원,20190925074803
2,20090831,휴업,0.0,20200801.0,20210731.0,서울특별시 강남구 논현동 59번지,서울특별시 강남구 도산대로 202 (논현동 강남을지병원),강남을지대학교병원,20200708095851
3,19910519,휴업,0.0,20190101.0,20211231.0,서울특별시 서대문구 북아현1동 126-22,서울특별시 서대문구 신촌로37길 10 (북아현동),의료법인 현창의료재단 서서울병원,20200709095440
4,19990602,휴업,0.0,20201005.0,20201011.0,서울특별시 강서구 염창동 263번지 3호,서울특별시 강서구 양천로 712 (염창동),강서연세병원,20200810180942
5,20081215,휴업,0.0,20200810.0,20201012.0,서울특별시 동작구 상도동 255번지 4호,서울특별시 동작구 성대로 43 (상도동 우리빌딩),서울요양병원,20200827151743
6,20150713,전출,0.0,0.0,0.0,서울특별시 서초구 서초동 1305번지 서산빌딩,서울특별시 서초구 강남대로 441 서산빌딩 11층 (서초동),라이브치과병원,20200902150234
7,20060411,전출,0.0,0.0,0.0,서울특별시 강남구 도곡동 942번지 3호,서울특별시 강남구 도곡로 122 (도곡동 삼우빌딩),기쁨병원,20200928140225


In [34]:
# 영업 종료했고 폐업일자가 null인 영화관의 폐업일자를 데이터 최종수정일자로 채우기
closed_null_list = (closed_null_df['최종수정일자']//1000000).values.tolist()
closed_null_list

[20081218.0,
 20081218.0,
 20090924.0,
 20171218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081215.0,
 20081215.0,
 20081215.0,
 20081215.0,
 20081215.0,
 20081215.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20081218.0,
 20110426.0,
 20081218.0,
 20081218.0,
 20100317.0]

In [35]:
closed_null_df['폐업일자'] = closed_null_list
closed_null_df['휴업시작일자'].fillna(0.0, inplace=True)
closed_null_df['휴업종료일자'].fillna(0.0, inplace=True)
closed_null_df['최종수정일자'] = closed_null_df['최종수정일자'].astype('int64')
closed_null_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,최종수정일자
0,20010326,폐업,20081218.0,0.0,0.0,서울특별시 영등포구 양평동1가 120-32,NaN,지성병원,20081218184441
1,20010427,폐업,20081218.0,0.0,0.0,서울특별시 영등포구 영등포동8가 12-1,NaN,서울성모병원,20081218184441
2,20051103,폐업,20090924.0,0.0,0.0,서울특별시 동대문구 전농로 37(답십리1동 266-3 2층 3층 267-8 1-3층),NaN,의료법인영일의료재단실로암요양병원,20090924085101
3,20080118,폐업,20171218.0,0.0,0.0,서울특별시 동대문구 고산자로 179(용두동 45-1) 지상2345층,NaN,효림병원,20171218151310
4,20010604,폐업,20081218.0,0.0,0.0,서울특별시 영등포구 신길동 103-21,NaN,서울대윤병원,20081218184441
5,20010801,폐업,20081218.0,0.0,0.0,서울특별시 영등포구 신길동 103-21,NaN,서울대윤병원,20081218184441
6,20010801,폐업,20081218.0,0.0,0.0,서울특별시 영등포구 당산동3가 386-3,NaN,영등포병원,20081218184441
7,20011029,폐업,20081218.0,0.0,0.0,서울특별시 영등포구 당산동3가 386-3,NaN,영등포병원,20081218184441
8,19000101,폐업,20081215.0,0.0,0.0,신수동345-2,NaN,서울광흥병원,20081215100318
9,19000101,폐업,20081215.0,0.0,0.0,성산동56-3,NaN,제일성모병원,20081215100319


In [36]:
notnull_df['휴업시작일자'].fillna(0.0, inplace=True)
notnull_df['휴업종료일자'].fillna(0.0, inplace=True)
notnull_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,최종수정일자
0,20080624,폐업,20181001.0,0.0,0.0,서울특별시 종로구 돈의동 78번지,서울특별시 종로구 돈화문로9길 26 (돈의동),춘원당한방병원,20181001134047
1,20080912,폐업,20111007.0,0.0,0.0,서울특별시 종로구 창신1동 327번지 2호 금호팰리스 4층,서울특별시 종로구 지봉로 29 (창신동),동대문요양병원,20111030144152
2,19820924,폐업,20130108.0,0.0,0.0,서울특별시 종로구 관훈동 196-10,NaN,해정병원,20130108143140
3,20060803,폐업,20120217.0,0.0,0.0,서울특별시 종로구 종로2가 75번지 9호,서울특별시 종로구 종로 78 (종로2가),밀레니엄치과병원,20120217111529
4,19660119,폐업,20080930.0,0.0,0.0,서울특별시 종로구 종로6가 70,NaN,이대부속동대문병원,20091019105921
...,...,...,...,...,...,...,...,...,...
237,20130201,폐업,20180119.0,0.0,0.0,서울특별시 강동구 천호동 357번지 56층,서울특별시 강동구 올림픽로 684 56층 (천호동),의료법인 한국암치료재단 헬스피아요양병원,20180119112523
238,20160923,폐업,20170701.0,0.0,0.0,NaN,서울특별시 강동구 양재대로 1412 345층 (둔촌동 토림빌),강동제일병원,20170628151701
239,20181101,폐업,20200817.0,0.0,0.0,서울특별시 강동구 길동 419번지,서울특별시 강동구 진황도로 122 (길동),서울실버한방병원,20200903141145
240,20070309,폐업,20200712.0,0.0,0.0,NaN,서울특별시 강동구 양재대로 1596 (명일동),러스크강동병원,20200713175745


In [37]:
# df 합치기
concat_df = pd.concat([open_null_df, closed_null_df, rest_null_df, notnull_df], axis=0).reset_index(drop=True)
concat_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,최종수정일자
0,19891012,영업중,0.0,0.0,0.0,NaN,서울특별시 종로구 삼청로10길 13 (삼청동),서울지구병원,20200228160711
1,20040901,영업중,0.0,0.0,0.0,NaN,서울특별시 종로구 대학로 101 (연건동),서울대학교치과병원,20190925074803
2,20190124,영업중,0.0,0.0,0.0,서울특별시 강동구 성내동 452번지 느루요양병원빌딩,서울특별시 강동구 강동대로 143-52 느루요양병원빌딩 (성내동),느루요양병원,20200708095851
3,19870424,영업중,0.0,0.0,0.0,서울특별시 종로구 무악동 32번지 2호,서울특별시 종로구 통일로 256 (무악동),세란병원,20200709095440
4,20200708,영업중,0.0,0.0,0.0,서울특별시 강서구 마곡동 799-11 파인스퀘어,서울특별시 강서구 마곡중앙4로 22 파인스퀘어 34층 (마곡동),보금한방병원,20200810180942
...,...,...,...,...,...,...,...,...,...
807,20130201,폐업,20180119.0,0.0,0.0,서울특별시 강동구 천호동 357번지 56층,서울특별시 강동구 올림픽로 684 56층 (천호동),의료법인 한국암치료재단 헬스피아요양병원,20180119112523
808,20160923,폐업,20170701.0,0.0,0.0,NaN,서울특별시 강동구 양재대로 1412 345층 (둔촌동 토림빌),강동제일병원,20170628151701
809,20181101,폐업,20200817.0,0.0,0.0,서울특별시 강동구 길동 419번지,서울특별시 강동구 진황도로 122 (길동),서울실버한방병원,20200903141145
810,20070309,폐업,20200712.0,0.0,0.0,NaN,서울특별시 강동구 양재대로 1596 (명일동),러스크강동병원,20200713175745


In [38]:
# 컬럼 타입 변환 : 폐업일자를 int64로 변환
concat_df = concat_df.astype({'폐업일자' : 'int64',
                             '휴업시작일자' : 'int64',
                             '휴업종료일자' : 'int64'})
concat_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,최종수정일자
0,19891012,영업중,0,0,0,NaN,서울특별시 종로구 삼청로10길 13 (삼청동),서울지구병원,20200228160711
1,20040901,영업중,0,0,0,NaN,서울특별시 종로구 대학로 101 (연건동),서울대학교치과병원,20190925074803
2,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452번지 느루요양병원빌딩,서울특별시 강동구 강동대로 143-52 느루요양병원빌딩 (성내동),느루요양병원,20200708095851
3,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32번지 2호,서울특별시 종로구 통일로 256 (무악동),세란병원,20200709095440
4,20200708,영업중,0,0,0,서울특별시 강서구 마곡동 799-11 파인스퀘어,서울특별시 강서구 마곡중앙4로 22 파인스퀘어 34층 (마곡동),보금한방병원,20200810180942
...,...,...,...,...,...,...,...,...,...
807,20130201,폐업,20180119,0,0,서울특별시 강동구 천호동 357번지 56층,서울특별시 강동구 올림픽로 684 56층 (천호동),의료법인 한국암치료재단 헬스피아요양병원,20180119112523
808,20160923,폐업,20170701,0,0,NaN,서울특별시 강동구 양재대로 1412 345층 (둔촌동 토림빌),강동제일병원,20170628151701
809,20181101,폐업,20200817,0,0,서울특별시 강동구 길동 419번지,서울특별시 강동구 진황도로 122 (길동),서울실버한방병원,20200903141145
810,20070309,폐업,20200712,0,0,NaN,서울특별시 강동구 양재대로 1596 (명일동),러스크강동병원,20200713175745


### 3. 데이터 분리

In [39]:
# 최종수정일자 컬럼 삭제
result_df = concat_df.drop('최종수정일자', axis=1)
result_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명
0,19891012,영업중,0,0,0,NaN,서울특별시 종로구 삼청로10길 13 (삼청동),서울지구병원
1,20040901,영업중,0,0,0,NaN,서울특별시 종로구 대학로 101 (연건동),서울대학교치과병원
2,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452번지 느루요양병원빌딩,서울특별시 강동구 강동대로 143-52 느루요양병원빌딩 (성내동),느루요양병원
3,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32번지 2호,서울특별시 종로구 통일로 256 (무악동),세란병원
4,20200708,영업중,0,0,0,서울특별시 강서구 마곡동 799-11 파인스퀘어,서울특별시 강서구 마곡중앙4로 22 파인스퀘어 34층 (마곡동),보금한방병원
...,...,...,...,...,...,...,...,...
807,20130201,폐업,20180119,0,0,서울특별시 강동구 천호동 357번지 56층,서울특별시 강동구 올림픽로 684 56층 (천호동),의료법인 한국암치료재단 헬스피아요양병원
808,20160923,폐업,20170701,0,0,NaN,서울특별시 강동구 양재대로 1412 345층 (둔촌동 토림빌),강동제일병원
809,20181101,폐업,20200817,0,0,서울특별시 강동구 길동 419번지,서울특별시 강동구 진황도로 122 (길동),서울실버한방병원
810,20070309,폐업,20200712,0,0,NaN,서울특별시 강동구 양재대로 1596 (명일동),러스크강동병원


#### 2017년 이후 데이터만 저장

In [42]:
# before_2017 = concat_df[(concat_df['폐업일자']!=0)&(concat_df['폐업일자']<=20170101)].reset_index(drop=True)
after_2017 = result_df[(result_df['폐업일자']==0)|(result_df['폐업일자']>20170101)].reset_index(drop=True)
after_2017

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명
0,19891012,영업중,0,0,0,NaN,서울특별시 종로구 삼청로10길 13 (삼청동),서울지구병원
1,20040901,영업중,0,0,0,NaN,서울특별시 종로구 대학로 101 (연건동),서울대학교치과병원
2,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452번지 느루요양병원빌딩,서울특별시 강동구 강동대로 143-52 느루요양병원빌딩 (성내동),느루요양병원
3,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32번지 2호,서울특별시 종로구 통일로 256 (무악동),세란병원
4,20200708,영업중,0,0,0,서울특별시 강서구 마곡동 799-11 파인스퀘어,서울특별시 강서구 마곡중앙4로 22 파인스퀘어 34층 (마곡동),보금한방병원
...,...,...,...,...,...,...,...,...
589,20130201,폐업,20180119,0,0,서울특별시 강동구 천호동 357번지 56층,서울특별시 강동구 올림픽로 684 56층 (천호동),의료법인 한국암치료재단 헬스피아요양병원
590,20160923,폐업,20170701,0,0,NaN,서울특별시 강동구 양재대로 1412 345층 (둔촌동 토림빌),강동제일병원
591,20181101,폐업,20200817,0,0,서울특별시 강동구 길동 419번지,서울특별시 강동구 진황도로 122 (길동),서울실버한방병원
592,20070309,폐업,20200712,0,0,NaN,서울특별시 강동구 양재대로 1596 (명일동),러스크강동병원


### 4. 지번주소 null값인 데이터 분리

In [43]:
# 지번주소 데이터
addr_df = after_2017[after_2017['지번주소'].notnull()].reset_index(drop=True)
addr_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명
0,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452번지 느루요양병원빌딩,서울특별시 강동구 강동대로 143-52 느루요양병원빌딩 (성내동),느루요양병원
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32번지 2호,서울특별시 종로구 통일로 256 (무악동),세란병원
2,20200708,영업중,0,0,0,서울특별시 강서구 마곡동 799-11 파인스퀘어,서울특별시 강서구 마곡중앙4로 22 파인스퀘어 34층 (마곡동),보금한방병원
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 108,서울특별시 종로구 새문안로 29 (평동),강북삼성병원
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505번지 2층~7층,서울특별시 영등포구 여의대방로 145 2~7층 (신길동 세인트빌딩),서울바른병원
...,...,...,...,...,...,...,...,...
487,19850329,폐업,20180119,0,0,서울특별시 강동구 천호2동 357번지,서울특별시 강동구 올림픽로 684 (천호동),의료법인 한국암치료재단 서울중입자병원
488,20051115,폐업,20170712,0,0,서울특별시 강동구 천호동 362번지 3호,서울특별시 강동구 올림픽로80길 31 (천호동),사회복지법인소원의항구로뎀노인전문병원
489,20130201,폐업,20180119,0,0,서울특별시 강동구 천호동 357번지 56층,서울특별시 강동구 올림픽로 684 56층 (천호동),의료법인 한국암치료재단 헬스피아요양병원
490,20181101,폐업,20200817,0,0,서울특별시 강동구 길동 419번지,서울특별시 강동구 진황도로 122 (길동),서울실버한방병원


In [44]:
# 지번주소 null 데이터
addr_null_df = after_2017[after_2017['지번주소'].isna()].reset_index(drop=True)
addr_null_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명
0,19891012,영업중,0,0,0,NaN,서울특별시 종로구 삼청로10길 13 (삼청동),서울지구병원
1,20040901,영업중,0,0,0,NaN,서울특별시 종로구 대학로 101 (연건동),서울대학교치과병원
2,19770401,영업중,0,0,0,NaN,서울특별시 종로구 새문안로 9 적십자병원 (평동),서울적십자병원
3,20060622,영업중,0,0,0,NaN,서울특별시 중구 장충단로 263 14층 (을지로6가 밀리오레상가건물),더블유치과병원
4,20111028,영업중,0,0,0,NaN,서울특별시 중구 퇴계로 447 211~213호 (황학동 황학아크로타워),연세스위트치과병원
...,...,...,...,...,...,...,...,...
97,20100902,폐업,20170807,0,0,NaN,서울특별시 중랑구 동일로 649 (면목동),해솔요양병원
98,20071231,폐업,20180501,0,0,NaN,서울특별시 강남구 학동로4길 7 (논현동 에스오케이병원 1~3층 4층일부),연세에스병원
99,20120912,폐업,20180516,0,0,NaN,서울특별시 송파구 마천로 238 (마천동 윤진프라자타워),송파마음요양병원
100,20160923,폐업,20170701,0,0,NaN,서울특별시 강동구 양재대로 1412 345층 (둔촌동 토림빌),강동제일병원


In [45]:
addr_df.to_csv('data/서울특별시 병원 인허가 정보(지번주소).csv', encoding='cp949', index=False)
addr_null_df.to_csv('data/서울특별시 병원 인허가 정보(지번주소null).csv', encoding='cp949', index=False)

### 5. GeoCoder 돌린 데이터 전처리
#### 1) 필요없는 컬럼 지우기

In [49]:
df1 = pd.read_csv('data/병원(지번주소).csv', encoding='cp949')
df2 = pd.read_csv('data/병원(지번주소2).csv', encoding='cp949')
df3 = pd.read_csv('data/병원(지번주소null).csv', encoding='cp949')

In [50]:
df1
df1.drop(['No', '상태'], axis=1, inplace=True)
df1

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도
0,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452번지 느루요양병원빌딩,서울특별시 강동구 강동대로 143-52 느루요양병원빌딩 (성내동),느루요양병원,127.120540,37.527865
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32번지 2호,서울특별시 종로구 통일로 256 (무악동),세란병원,126.957451,37.575170
2,20200708,영업중,0,0,0,서울특별시 강서구 마곡동 799-11 파인스퀘어,서울특별시 강서구 마곡중앙4로 22 파인스퀘어 34층 (마곡동),보금한방병원,126.819730,37.569973
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 108,서울특별시 종로구 새문안로 29 (평동),강북삼성병원,126.966701,37.568222
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505번지 2층~7층,서울특별시 영등포구 여의대방로 145 2~7층 (신길동 세인트빌딩),서울바른병원,126.919996,37.500298
...,...,...,...,...,...,...,...,...,...,...
459,20100610,폐업,20190812,0,0,서울특별시 강동구 길동 412번지 10호,서울특별시 강동구 양재대로 1461 (길동),예닮요양병원,127.139019,37.536955
460,20051115,폐업,20170712,0,0,서울특별시 강동구 천호동 362번지 3호,서울특별시 강동구 올림픽로80길 31 (천호동),사회복지법인소원의항구로뎀노인전문병원,127.127336,37.543887
461,20130201,폐업,20180119,0,0,서울특별시 강동구 천호동 357번지 56층,서울특별시 강동구 올림픽로 684 56층 (천호동),의료법인 한국암치료재단 헬스피아요양병원,127.125282,37.542259
462,20181101,폐업,20200817,0,0,서울특별시 강동구 길동 419번지,서울특별시 강동구 진황도로 122 (길동),서울실버한방병원,127.139414,37.534549


In [51]:
df2
df2.drop(['No', 'No.1', '상태', '경도.1', '위도.1', '상태.1'], axis=1, inplace=True)
df2

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도
0,20090603,영업중,0,0,0,서울특별시 성동구 성수2가3동 275번지 2호,서울특별시 성동구 동일로 93 (성수동2가),서울효사랑요양병원,127.063099,37.541147
1,20101006,영업중,0,0,0,서울특별시 성동구 행당1동 198번지 1호,서울특별시 성동구 왕십리로 296 (행당동),서울연세병원,127.036148,37.560640
2,20070528,영업중,0,0,0,서울특별시 광진구 구의2동 80번지 25호,서울특별시 광진구 자양로 215 (구의동),제니스병원,127.086448,37.546514
3,19660107,영업중,0,0,0,서울특별시 동대문구 휘경2동 29번지 1호 산 6번지 2호외 7필지,서울특별시 동대문구 망우로 82 (휘경동),삼육서울병원,127.062542,37.589726
4,19710821,영업중,0,0,0,서울특별시 동대문구 경희대길 45(회기동 산1번지),서울특별시 동대문구 경희대로 23 (회기동),경희대학교한방병원,127.050740,37.593774
5,20110325,영업중,0,0,0,서울특별시 동대문구 장안2동 294번지 11호 369-3(6~9층),서울특별시 동대문구 답십리로 261 (장안동),참튼튼병원,127.069359,37.572732
6,19921112,영업중,0,0,0,서울특별시 동대문구 휘경2동 20번지 산6-7,서울특별시 동대문구 망우로 82 (휘경동),삼육치과병원,127.062542,37.589726
7,20051207,영업중,0,0,0,서울시 성북구 장위3동 50-50,서울특별시 성북구 화랑로 271 (장위동),의료법인유라의료재단 온누리요양병원,127.058809,37.612171
8,19930203,영업중,0,0,0,서울특별시 도봉구 705번지 4호 6호2호7호,서울특별시 도봉구 도봉로 720 (방학동),도봉병원,127.043802,37.668760
9,20040823,영업중,0,0,0,서울특별시 도봉구 방학3동 442,서울특별시 도봉구 시루봉로 137 (방학동),경희늘푸른노인전문병원,127.027370,37.663194


In [52]:
df3
df3.drop(['No', '상태'], axis=1, inplace=True)
df3

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도
0,19891012,영업중,0,0,0,NaN,서울특별시 종로구 삼청로10길 13 (삼청동),서울지구병원,126.982009,37.587868
1,20040901,영업중,0,0,0,NaN,서울특별시 종로구 대학로 101 (연건동),서울대학교치과병원,126.998354,37.578547
2,19770401,영업중,0,0,0,NaN,서울특별시 종로구 새문안로 9 적십자병원 (평동),서울적십자병원,126.967386,37.567497
3,20060622,영업중,0,0,0,NaN,서울특별시 중구 장충단로 263 14층 (을지로6가 밀리오레상가건물),더블유치과병원,127.008472,37.568115
4,20111028,영업중,0,0,0,NaN,서울특별시 중구 퇴계로 447 211~213호 (황학동 황학아크로타워),연세스위트치과병원,127.021722,37.566042
...,...,...,...,...,...,...,...,...,...,...
97,20100902,폐업,20170807,0,0,NaN,서울특별시 중랑구 동일로 649 (면목동),해솔요양병원,127.079709,37.588459
98,20071231,폐업,20180501,0,0,NaN,서울특별시 강남구 학동로4길 7 (논현동 에스오케이병원 1~3층 4층일부),연세에스병원,127.023176,37.511041
99,20120912,폐업,20180516,0,0,NaN,서울특별시 송파구 마천로 238 (마천동 윤진프라자타워),송파마음요양병원,127.143772,37.499639
100,20160923,폐업,20170701,0,0,NaN,서울특별시 강동구 양재대로 1412 345층 (둔촌동 토림빌),강동제일병원,127.138323,37.532011


In [53]:
fin_df = pd.concat([df1, df2, df3], axis=0).reset_index(drop=True)
fin_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도
0,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452번지 느루요양병원빌딩,서울특별시 강동구 강동대로 143-52 느루요양병원빌딩 (성내동),느루요양병원,127.120540,37.527865
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32번지 2호,서울특별시 종로구 통일로 256 (무악동),세란병원,126.957451,37.575170
2,20200708,영업중,0,0,0,서울특별시 강서구 마곡동 799-11 파인스퀘어,서울특별시 강서구 마곡중앙4로 22 파인스퀘어 34층 (마곡동),보금한방병원,126.819730,37.569973
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 108,서울특별시 종로구 새문안로 29 (평동),강북삼성병원,126.966701,37.568222
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505번지 2층~7층,서울특별시 영등포구 여의대방로 145 2~7층 (신길동 세인트빌딩),서울바른병원,126.919996,37.500298
...,...,...,...,...,...,...,...,...,...,...
589,20100902,폐업,20170807,0,0,NaN,서울특별시 중랑구 동일로 649 (면목동),해솔요양병원,127.079709,37.588459
590,20071231,폐업,20180501,0,0,NaN,서울특별시 강남구 학동로4길 7 (논현동 에스오케이병원 1~3층 4층일부),연세에스병원,127.023176,37.511041
591,20120912,폐업,20180516,0,0,NaN,서울특별시 송파구 마천로 238 (마천동 윤진프라자타워),송파마음요양병원,127.143772,37.499639
592,20160923,폐업,20170701,0,0,NaN,서울특별시 강동구 양재대로 1412 345층 (둔촌동 토림빌),강동제일병원,127.138323,37.532011


In [54]:
fin_df.to_csv('data/병원(좌표추가).csv', encoding='cp949', index=False)

### 6. 주소 생성한 데이터 전처리
#### 1) 필요없는 컬럼 삭제

In [3]:
new_addr_df = pd.read_csv('data/병원/병원(주소추가).csv', encoding='cp949')
new_addr_df

,No,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,경도.1,지번주소.1,도로명주소.1
0,1,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452,서울특별시 강동구 강동대로 143-52,느루요양병원,127.120540,37.527865,NaN,NaN,NaN
1,2,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,세란병원,126.957451,37.575170,NaN,NaN,NaN
2,3,20200708,영업중,0,0,0,서울특별시 강서구 마곡동 731-11,서울특별시 강서구 방화대로34길 60,보금한방병원,126.819730,37.569973,NaN,NaN,NaN
3,4,19790324,영업중,0,0,0,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,강북삼성병원,126.966701,37.568222,NaN,NaN,NaN
4,5,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,서울바른병원,126.919996,37.500298,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,590,20100902,폐업,20170807,0,0,서울특별시 중랑구 면목동 179-21,서울특별시 중랑구 동일로 649,해솔요양병원,127.079709,37.588459,NaN,NaN,NaN
590,591,20071231,폐업,20180501,0,0,서울특별시 강남구 논현동 122-5,서울특별시 강남구 학동로4길 7,연세에스병원,127.023176,37.511041,NaN,NaN,NaN
591,592,20120912,폐업,20180516,0,0,서울특별시 송파구 마천동 32,서울특별시 송파구 마천로 238,송파마음요양병원,127.143772,37.499639,NaN,NaN,NaN
592,593,20160923,폐업,20170701,0,0,서울특별시 강동구 둔촌동 439-5,서울특별시 강동구 양재대로 1412,강동제일병원,127.138323,37.532011,NaN,NaN,NaN


In [4]:
new_addr_df.drop(['No', '경도.1', '지번주소.1', '도로명주소.1'], axis=1, inplace=True)
new_addr_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도
0,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452,서울특별시 강동구 강동대로 143-52,느루요양병원,127.120540,37.527865
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,세란병원,126.957451,37.575170
2,20200708,영업중,0,0,0,서울특별시 강서구 마곡동 731-11,서울특별시 강서구 방화대로34길 60,보금한방병원,126.819730,37.569973
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,강북삼성병원,126.966701,37.568222
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,서울바른병원,126.919996,37.500298
...,...,...,...,...,...,...,...,...,...,...
589,20100902,폐업,20170807,0,0,서울특별시 중랑구 면목동 179-21,서울특별시 중랑구 동일로 649,해솔요양병원,127.079709,37.588459
590,20071231,폐업,20180501,0,0,서울특별시 강남구 논현동 122-5,서울특별시 강남구 학동로4길 7,연세에스병원,127.023176,37.511041
591,20120912,폐업,20180516,0,0,서울특별시 송파구 마천동 32,서울특별시 송파구 마천로 238,송파마음요양병원,127.143772,37.499639
592,20160923,폐업,20170701,0,0,서울특별시 강동구 둔촌동 439-5,서울특별시 강동구 양재대로 1412,강동제일병원,127.138323,37.532011


In [5]:
new_addr_df.isna().sum()

인허가일자      0
상세영업상태명    0
폐업일자       0
휴업시작일자     0
휴업종료일자     0
지번주소       0
도로명주소      1
사업장명       0
경도         0
위도         0
dtype: int64

#### 2) 2017년 영업 여부 수정

In [89]:
df_2017 = new_addr_df[(new_addr_df['폐업일자']==0) | (new_addr_df['폐업일자']>20170101)]
df_2017 = df_2017[df_2017['인허가일자']<20180101]
df_2017['년'] = 2017
df_2017

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,세란병원,126.957451,37.575170,2017
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,강북삼성병원,126.966701,37.568222,2017
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,서울바른병원,126.919996,37.500298,2017
5,20000810,영업중,0,0,0,서울특별시 중구 충무로1가 26-1,서울특별시 중구 충무로1가 명동2길 50,한누리치과병원,126.983100,37.561179,2017
6,20160708,영업중,0,0,0,서울특별시 중구 순화동 1-1,서울특별시 중구 통일로4길 30,광화문자생한방병원,126.969859,37.564869,2017
...,...,...,...,...,...,...,...,...,...,...,...
589,20100902,폐업,20170807,0,0,서울특별시 중랑구 면목동 179-21,서울특별시 중랑구 동일로 649,해솔요양병원,127.079709,37.588459,2017
590,20071231,폐업,20180501,0,0,서울특별시 강남구 논현동 122-5,서울특별시 강남구 학동로4길 7,연세에스병원,127.023176,37.511041,2017
591,20120912,폐업,20180516,0,0,서울특별시 송파구 마천동 32,서울특별시 송파구 마천로 238,송파마음요양병원,127.143772,37.499639,2017
592,20160923,폐업,20170701,0,0,서울특별시 강동구 둔촌동 439-5,서울특별시 강동구 양재대로 1412,강동제일병원,127.138323,37.532011,2017


In [94]:
# 전출 row 삭제
idx = df_2017[df_2017['상세영업상태명']=='전출'].index
df_2017.drop(idx, axis=0, inplace=True)
df_2017

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,세란병원,126.957451,37.575170,2017
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,강북삼성병원,126.966701,37.568222,2017
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,서울바른병원,126.919996,37.500298,2017
5,20000810,영업중,0,0,0,서울특별시 중구 충무로1가 26-1,서울특별시 중구 충무로1가 명동2길 50,한누리치과병원,126.983100,37.561179,2017
6,20160708,영업중,0,0,0,서울특별시 중구 순화동 1-1,서울특별시 중구 통일로4길 30,광화문자생한방병원,126.969859,37.564869,2017
...,...,...,...,...,...,...,...,...,...,...,...
589,20100902,폐업,20170807,0,0,서울특별시 중랑구 면목동 179-21,서울특별시 중랑구 동일로 649,해솔요양병원,127.079709,37.588459,2017
590,20071231,폐업,20180501,0,0,서울특별시 강남구 논현동 122-5,서울특별시 강남구 학동로4길 7,연세에스병원,127.023176,37.511041,2017
591,20120912,폐업,20180516,0,0,서울특별시 송파구 마천동 32,서울특별시 송파구 마천로 238,송파마음요양병원,127.143772,37.499639,2017
592,20160923,폐업,20170701,0,0,서울특별시 강동구 둔촌동 439-5,서울특별시 강동구 양재대로 1412,강동제일병원,127.138323,37.532011,2017


#### 휴업 처리

In [96]:
rest_df = df_2017[(df_2017['휴업시작일자']!=0)&(df_2017['폐업일자']==0)].reset_index(drop=True)
not_open_df = df_2017[(df_2017['휴업시작일자']!=0)&(df_2017['폐업일자']!=0)].reset_index(drop=True)
still_df = df_2017[df_2017['휴업시작일자']==0].reset_index(drop=True)
print(len(rest_df)+len(not_open_df)+len(still_df))

519


In [97]:
not_open_df['휴업시작일자'] = 0
not_open_df['휴업종료일자'] = 0
not_open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20110603,폐업,20190328,0,0,서울특별시 노원구 상계동 366-1,서울특별시 노원구 노원로 466,노원스마트요양병원,127.065982,37.657992,2017


In [98]:
# 휴업시작일자가 20180101 이후일 경우
rest_df['상세영업상태명'] = '영업중'
rest_df['휴업시작일자'] = 0
rest_df['휴업종료일자'] = 0
rest_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20110602,영업중,0,0,0,서울특별시 중구 을지로3가 302-1,서울특별시 중구 을지로3가 을지로14길 16-6,을지치과병원,126.991866,37.565585,2017
1,20090831,영업중,0,0,0,서울특별시 강남구 논현동 59,서울특별시 강남구 도산대로 202,강남을지대학교병원,127.028560,37.519400,2017
2,19990602,영업중,0,0,0,서울특별시 강서구 염창동 263,서울특별시 강서구 양천로 706,강서연세병원,126.874000,37.549119,2017
3,20081215,영업중,0,0,0,서울특별시 동작구 상도동 255,서울특별시 동작구 성대로 31,서울요양병원,126.933770,37.499142,2017
4,19910519,영업중,0,0,0,서울특별시 서대문구 북아현동 126-22,서울특별시 서대문구 신촌로37길 10,의료법인 현창의료재단 서서울병원,126.957006,37.557718,2017
5,20170106,영업중,0,0,0,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.028020,37.565111,2017


In [99]:
df_2017 = pd.concat([rest_df, not_open_df, still_df], axis=0).reset_index(drop=True)
df_2017.sort_values(by=['인허가일자'], axis=0, inplace=True)

In [100]:
open_df = df_2017[df_2017['폐업일자'] >= 20180101].reset_index(drop=True)
closed_df = df_2017[(df_2017['폐업일자'] == 0) |  (df_2017['폐업일자'] < 20180101)].reset_index(drop=True)
open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,19660107,폐업,20180331,0,0,서울특별시 동대문구 청량리동 46-3,서울특별시 동대문구 제기로 144,청량리정신병원,127.049088,37.586070,2017
1,19850329,폐업,20180119,0,0,서울특별시 강동구 천호동 357,서울특별시 강동구 올림픽로 684,의료법인 한국암치료재단 서울중입자병원,127.125299,37.542277,2017
2,19850422,폐업,20181101,0,0,서울특별시 성동구 금호동2가 500,서울특별시 성동구 금호동2가 금호산15길 5,서울중앙병원,127.018774,37.553606,2017
3,19901222,폐업,20180603,0,0,서울특별시 강남구 역삼동 648,서울특별시 강남구 테헤란로7길 12,청아치과병원,127.030921,37.500017,2017
4,20000104,폐업,20180402,0,0,서울특별시 영등포구 대림동 706,서울특별시 영등포구 도림로 142,서울복지병원,126.897877,37.493169,2017
5,20000328,폐업,20180705,0,0,서울특별시 강서구 방화동 487-5,서울특별시 강서구 양천로 45-5,뉴강서성심병원,126.808801,37.573288,2017
6,20001220,폐업,20180430,0,0,서울특별시 영등포구 대림동 706-1,서울특별시 영등포구 도림로 142,서울복지한방병원,126.898078,37.493323,2017
7,20020104,폐업,20190117,0,0,서울특별시 성북구 장위동 33-1,서울특별시 성북구 한천로 638,의료법인조광의료재단 주사랑병원,127.058143,37.615317,2017
8,20031101,폐업,20190504,0,0,서울특별시 영등포구 여의도동 26,서울특별시 영등포구 국제금융로2길 37,예림치과병원,126.924076,37.522888,2017
9,20050420,폐업,20180514,0,0,서울특별시 노원구 상계동 301-6,서울특별시 노원구 노원로 416,모네여성병원,127.067870,37.653704,2017


In [101]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업중'
open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,19660107,영업중,0,0,0,서울특별시 동대문구 청량리동 46-3,서울특별시 동대문구 제기로 144,청량리정신병원,127.049088,37.586070,2017
1,19850329,영업중,0,0,0,서울특별시 강동구 천호동 357,서울특별시 강동구 올림픽로 684,의료법인 한국암치료재단 서울중입자병원,127.125299,37.542277,2017
2,19850422,영업중,0,0,0,서울특별시 성동구 금호동2가 500,서울특별시 성동구 금호동2가 금호산15길 5,서울중앙병원,127.018774,37.553606,2017
3,19901222,영업중,0,0,0,서울특별시 강남구 역삼동 648,서울특별시 강남구 테헤란로7길 12,청아치과병원,127.030921,37.500017,2017
4,20000104,영업중,0,0,0,서울특별시 영등포구 대림동 706,서울특별시 영등포구 도림로 142,서울복지병원,126.897877,37.493169,2017
5,20000328,영업중,0,0,0,서울특별시 강서구 방화동 487-5,서울특별시 강서구 양천로 45-5,뉴강서성심병원,126.808801,37.573288,2017
6,20001220,영업중,0,0,0,서울특별시 영등포구 대림동 706-1,서울특별시 영등포구 도림로 142,서울복지한방병원,126.898078,37.493323,2017
7,20020104,영업중,0,0,0,서울특별시 성북구 장위동 33-1,서울특별시 성북구 한천로 638,의료법인조광의료재단 주사랑병원,127.058143,37.615317,2017
8,20031101,영업중,0,0,0,서울특별시 영등포구 여의도동 26,서울특별시 영등포구 국제금융로2길 37,예림치과병원,126.924076,37.522888,2017
9,20050420,영업중,0,0,0,서울특별시 노원구 상계동 301-6,서울특별시 노원구 노원로 416,모네여성병원,127.067870,37.653704,2017


In [102]:
fin_2017_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2017_df.sort_values(by=['인허가일자'], axis=0, inplace=True)

#### 3) 2018년 영업 여부 수정

In [105]:
df_2018 = new_addr_df[(new_addr_df['폐업일자']==0) | (new_addr_df['폐업일자']>20180101)]
df_2018 = df_2018[df_2018['인허가일자']<20190101]
df_2018['년'] = 2018
df_2018

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,세란병원,126.957451,37.575170,2018
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,강북삼성병원,126.966701,37.568222,2018
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,서울바른병원,126.919996,37.500298,2018
5,20000810,영업중,0,0,0,서울특별시 중구 충무로1가 26-1,서울특별시 중구 충무로1가 명동2길 50,한누리치과병원,126.983100,37.561179,2018
6,20160708,영업중,0,0,0,서울특별시 중구 순화동 1-1,서울특별시 중구 통일로4길 30,광화문자생한방병원,126.969859,37.564869,2018
...,...,...,...,...,...,...,...,...,...,...,...
586,19710220,영업중,0,0,0,서울특별시 송파구 가락동 58,서울특별시 송파구 송이로 123,경찰병원,127.123487,37.496415,2018
587,20170106,휴업,0,20200801,20201231,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.028020,37.565111,2018
590,20071231,폐업,20180501,0,0,서울특별시 강남구 논현동 122-5,서울특별시 강남구 학동로4길 7,연세에스병원,127.023176,37.511041,2018
591,20120912,폐업,20180516,0,0,서울특별시 송파구 마천동 32,서울특별시 송파구 마천로 238,송파마음요양병원,127.143772,37.499639,2018


In [106]:
# 전출 row 삭제
idx = df_2018[df_2018['상세영업상태명']=='전출'].index
df_2018.drop(idx, axis=0, inplace=True)
df_2018

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,세란병원,126.957451,37.575170,2018
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,강북삼성병원,126.966701,37.568222,2018
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,서울바른병원,126.919996,37.500298,2018
5,20000810,영업중,0,0,0,서울특별시 중구 충무로1가 26-1,서울특별시 중구 충무로1가 명동2길 50,한누리치과병원,126.983100,37.561179,2018
6,20160708,영업중,0,0,0,서울특별시 중구 순화동 1-1,서울특별시 중구 통일로4길 30,광화문자생한방병원,126.969859,37.564869,2018
...,...,...,...,...,...,...,...,...,...,...,...
586,19710220,영업중,0,0,0,서울특별시 송파구 가락동 58,서울특별시 송파구 송이로 123,경찰병원,127.123487,37.496415,2018
587,20170106,휴업,0,20200801,20201231,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.028020,37.565111,2018
590,20071231,폐업,20180501,0,0,서울특별시 강남구 논현동 122-5,서울특별시 강남구 학동로4길 7,연세에스병원,127.023176,37.511041,2018
591,20120912,폐업,20180516,0,0,서울특별시 송파구 마천동 32,서울특별시 송파구 마천로 238,송파마음요양병원,127.143772,37.499639,2018


In [107]:
rest_df = df_2018[(df_2018['휴업시작일자']!=0)&(df_2018['폐업일자']==0)].reset_index(drop=True)
not_open_df = df_2018[(df_2018['휴업시작일자']!=0)&(df_2018['폐업일자']!=0)].reset_index(drop=True)
still_df = df_2018[df_2018['휴업시작일자']==0].reset_index(drop=True)
print(len(rest_df)+len(not_open_df)+len(still_df))

528


In [110]:
not_open_df['휴업시작일자'] = 0
not_open_df['휴업종료일자'] = 0
not_open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20110603,폐업,20190328,0,0,서울특별시 노원구 상계동 366-1,서울특별시 노원구 노원로 466,노원스마트요양병원,127.065982,37.657992,2018


In [112]:
# 휴업시작일자가 20190101 이후일 경우
rest_df['상세영업상태명'] = '영업중'
rest_df['휴업시작일자'] = 0
rest_df['휴업종료일자'] = 0
rest_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20110602,영업중,0,0,0,서울특별시 중구 을지로3가 302-1,서울특별시 중구 을지로3가 을지로14길 16-6,을지치과병원,126.991866,37.565585,2018
1,20090831,영업중,0,0,0,서울특별시 강남구 논현동 59,서울특별시 강남구 도산대로 202,강남을지대학교병원,127.028560,37.519400,2018
2,19990602,영업중,0,0,0,서울특별시 강서구 염창동 263,서울특별시 강서구 양천로 706,강서연세병원,126.874000,37.549119,2018
3,20081215,영업중,0,0,0,서울특별시 동작구 상도동 255,서울특별시 동작구 성대로 31,서울요양병원,126.933770,37.499142,2018
4,19910519,영업중,0,0,0,서울특별시 서대문구 북아현동 126-22,서울특별시 서대문구 신촌로37길 10,의료법인 현창의료재단 서서울병원,126.957006,37.557718,2018
5,20170106,영업중,0,0,0,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.028020,37.565111,2018


In [113]:
df_2018 = pd.concat([rest_df, not_open_df, still_df], axis=0).reset_index(drop=True)
df_2018.sort_values(by=['인허가일자'], axis=0, inplace=True)

In [114]:
open_df = df_2018[df_2018['폐업일자'] >= 20190101].reset_index(drop=True)
closed_df = df_2018[(df_2018['폐업일자'] == 0) | (df_2018['폐업일자'] < 20190101)].reset_index(drop=True)
open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20020104,폐업,20190117,0,0,서울특별시 성북구 장위동 33-1,서울특별시 성북구 한천로 638,의료법인조광의료재단 주사랑병원,127.058143,37.615317,2018
1,20031101,폐업,20190504,0,0,서울특별시 영등포구 여의도동 26,서울특별시 영등포구 국제금융로2길 37,예림치과병원,126.924076,37.522888,2018
2,20050610,폐업,20200723,0,0,서울특별시 동대문구 청량리동 689,서울특별시 동대문구 약령시로 124,경동요양병원,127.043743,37.582935,2018
3,20060519,폐업,20200828,0,0,서울특별시 강남구 대치동 889,서울특별시 강남구 선릉로 435,네모치과병원,127.049331,37.504219,2018
4,20070309,폐업,20200712,0,0,서울특별시 강동구 명일동 335-3,서울특별시 강동구 양재대로 1596,러스크강동병원,127.143790,37.548084,2018
5,20071005,폐업,20190115,0,0,서울특별시 도봉구 쌍문동 507-43,서울특별시 도봉구 삼양로 598,조이요양병원,127.014076,37.656243,2018
6,20100610,폐업,20190812,0,0,서울특별시 강동구 길동 412-1,서울특별시 강동구 양재대로111길 7,예닮요양병원,127.139019,37.536955,2018
7,20110211,폐업,20200314,0,0,서울특별시 양천구 신월동 549-1,서울특별시 양천구 신월로 150,서울인화요양병원,126.840480,37.517063,2018
8,20110603,폐업,20190328,0,0,서울특별시 노원구 상계동 366-1,서울특별시 노원구 노원로 466,노원스마트요양병원,127.065982,37.657992,2018
9,20110808,폐업,20190528,0,0,서울특별시 도봉구 방학동 677,서울특별시 도봉구 방학로 101,(사)한국지체장애인협회 새보람요양병원,127.041005,37.662593,2018


In [115]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업중'
open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20020104,영업중,0,0,0,서울특별시 성북구 장위동 33-1,서울특별시 성북구 한천로 638,의료법인조광의료재단 주사랑병원,127.058143,37.615317,2018
1,20031101,영업중,0,0,0,서울특별시 영등포구 여의도동 26,서울특별시 영등포구 국제금융로2길 37,예림치과병원,126.924076,37.522888,2018
2,20050610,영업중,0,0,0,서울특별시 동대문구 청량리동 689,서울특별시 동대문구 약령시로 124,경동요양병원,127.043743,37.582935,2018
3,20060519,영업중,0,0,0,서울특별시 강남구 대치동 889,서울특별시 강남구 선릉로 435,네모치과병원,127.049331,37.504219,2018
4,20070309,영업중,0,0,0,서울특별시 강동구 명일동 335-3,서울특별시 강동구 양재대로 1596,러스크강동병원,127.143790,37.548084,2018
5,20071005,영업중,0,0,0,서울특별시 도봉구 쌍문동 507-43,서울특별시 도봉구 삼양로 598,조이요양병원,127.014076,37.656243,2018
6,20100610,영업중,0,0,0,서울특별시 강동구 길동 412-1,서울특별시 강동구 양재대로111길 7,예닮요양병원,127.139019,37.536955,2018
7,20110211,영업중,0,0,0,서울특별시 양천구 신월동 549-1,서울특별시 양천구 신월로 150,서울인화요양병원,126.840480,37.517063,2018
8,20110603,영업중,0,0,0,서울특별시 노원구 상계동 366-1,서울특별시 노원구 노원로 466,노원스마트요양병원,127.065982,37.657992,2018
9,20110808,영업중,0,0,0,서울특별시 도봉구 방학동 677,서울특별시 도봉구 방학로 101,(사)한국지체장애인협회 새보람요양병원,127.041005,37.662593,2018


In [116]:
fin_2018_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2018_df.sort_values(by=['인허가일자'], axis=0, inplace=True)

#### 4) 2019년 영업 여부 수정

In [117]:
df_2019 = new_addr_df[(new_addr_df['폐업일자']==0) | (new_addr_df['폐업일자']>20190101)]
df_2019 = df_2019[df_2019['인허가일자']<20200101]
df_2019['년'] = 2019
df_2019

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452,서울특별시 강동구 강동대로 143-52,느루요양병원,127.120540,37.527865,2019
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,세란병원,126.957451,37.575170,2019
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,강북삼성병원,126.966701,37.568222,2019
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,서울바른병원,126.919996,37.500298,2019
5,20000810,영업중,0,0,0,서울특별시 중구 충무로1가 26-1,서울특별시 중구 충무로1가 명동2길 50,한누리치과병원,126.983100,37.561179,2019
...,...,...,...,...,...,...,...,...,...,...,...
584,19840904,영업중,0,0,0,서울특별시 서대문구 홍제동 234-2,서울특별시 서대문구 세무서길 20,재단법인대성병원,126.945792,37.588938,2019
585,19740325,영업중,0,0,0,서울특별시 서대문구 신촌동 134,서울특별시 서대문구 연세로 50-1,학교법인연세대학교의과대학세브란스병원,126.941721,37.563013,2019
586,19710220,영업중,0,0,0,서울특별시 송파구 가락동 58,서울특별시 송파구 송이로 123,경찰병원,127.123487,37.496415,2019
587,20170106,휴업,0,20200801,20201231,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.028020,37.565111,2019


In [119]:
# 전출 row 삭제
idx = df_2019[df_2019['상세영업상태명']=='전출'].index
df_2019.drop(idx, axis=0, inplace=True)
df_2019

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20190124,영업중,0,0,0,서울특별시 강동구 성내동 452,서울특별시 강동구 강동대로 143-52,느루요양병원,127.120540,37.527865,2019
1,19870424,영업중,0,0,0,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,세란병원,126.957451,37.575170,2019
3,19790324,영업중,0,0,0,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,강북삼성병원,126.966701,37.568222,2019
4,20151214,영업중,0,0,0,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,서울바른병원,126.919996,37.500298,2019
5,20000810,영업중,0,0,0,서울특별시 중구 충무로1가 26-1,서울특별시 중구 충무로1가 명동2길 50,한누리치과병원,126.983100,37.561179,2019
...,...,...,...,...,...,...,...,...,...,...,...
584,19840904,영업중,0,0,0,서울특별시 서대문구 홍제동 234-2,서울특별시 서대문구 세무서길 20,재단법인대성병원,126.945792,37.588938,2019
585,19740325,영업중,0,0,0,서울특별시 서대문구 신촌동 134,서울특별시 서대문구 연세로 50-1,학교법인연세대학교의과대학세브란스병원,126.941721,37.563013,2019
586,19710220,영업중,0,0,0,서울특별시 송파구 가락동 58,서울특별시 송파구 송이로 123,경찰병원,127.123487,37.496415,2019
587,20170106,휴업,0,20200801,20201231,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.028020,37.565111,2019


In [120]:
rest_df = df_2019[(df_2019['휴업시작일자']!=0)&(df_2019['폐업일자']==0)].reset_index(drop=True)
not_open_df = df_2019[(df_2019['휴업시작일자']!=0)&(df_2019['폐업일자']!=0)].reset_index(drop=True)
still_df = df_2019[df_2019['휴업시작일자']==0].reset_index(drop=True)
print(len(rest_df)+len(not_open_df)+len(still_df))

537


In [121]:
not_open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20110603,폐업,20190328,20190228,20190527,서울특별시 노원구 상계동 366-1,서울특별시 노원구 노원로 466,노원스마트요양병원,127.065982,37.657992,2019


In [122]:
not_open_df['휴업시작일자'] = 0
not_open_df['휴업종료일자'] = 0
not_open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20110603,폐업,20190328,0,0,서울특별시 노원구 상계동 366-1,서울특별시 노원구 노원로 466,노원스마트요양병원,127.065982,37.657992,2019


In [123]:
rest_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20110602,휴업,0,20191120,20201119,서울특별시 중구 을지로3가 302-1,서울특별시 중구 을지로3가 을지로14길 16-6,을지치과병원,126.991866,37.565585,2019
1,20090831,휴업,0,20200801,20210731,서울특별시 강남구 논현동 59,서울특별시 강남구 도산대로 202,강남을지대학교병원,127.028560,37.519400,2019
2,19990602,휴업,0,20201005,20201011,서울특별시 강서구 염창동 263,서울특별시 강서구 양천로 706,강서연세병원,126.874000,37.549119,2019
3,20081215,휴업,0,20200810,20201012,서울특별시 동작구 상도동 255,서울특별시 동작구 성대로 31,서울요양병원,126.933770,37.499142,2019
4,19910519,휴업,0,20190101,20211231,서울특별시 서대문구 북아현동 126-22,서울특별시 서대문구 신촌로37길 10,의료법인 현창의료재단 서서울병원,126.957006,37.557718,2019
5,20170106,휴업,0,20200801,20201231,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.028020,37.565111,2019


In [128]:
not_rest_df_2019 = rest_df[rest_df['휴업시작일자']>=20200101].reset_index(drop=True)
rest_df_2019 = rest_df[rest_df['휴업시작일자']<20200101].reset_index(drop=True)
not_rest_df_2019

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20090831,휴업,0,20200801,20210731,서울특별시 강남구 논현동 59,서울특별시 강남구 도산대로 202,강남을지대학교병원,127.02856,37.519400,2019
1,19990602,휴업,0,20201005,20201011,서울특별시 강서구 염창동 263,서울특별시 강서구 양천로 706,강서연세병원,126.87400,37.549119,2019
2,20081215,휴업,0,20200810,20201012,서울특별시 동작구 상도동 255,서울특별시 동작구 성대로 31,서울요양병원,126.93377,37.499142,2019
3,20170106,휴업,0,20200801,20201231,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.02802,37.565111,2019


In [129]:
# 휴업시작일자가 20200101 이후일 경우
not_rest_df_2019['상세영업상태명'] = '영업중'
not_rest_df_2019['휴업시작일자'] = 0
not_rest_df_2019['휴업종료일자'] = 0
not_rest_df_2019

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20090831,영업중,0,0,0,서울특별시 강남구 논현동 59,서울특별시 강남구 도산대로 202,강남을지대학교병원,127.02856,37.519400,2019
1,19990602,영업중,0,0,0,서울특별시 강서구 염창동 263,서울특별시 강서구 양천로 706,강서연세병원,126.87400,37.549119,2019
2,20081215,영업중,0,0,0,서울특별시 동작구 상도동 255,서울특별시 동작구 성대로 31,서울요양병원,126.93377,37.499142,2019
3,20170106,영업중,0,0,0,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,서울동인병원,127.02802,37.565111,2019


In [130]:
rest_df = pd.concat([rest_df_2019, not_rest_df_2019], axis=0).reset_index(drop=True)
rest_df.sort_values(by=['인허가일자'], axis=0, inplace=True)

In [131]:
df_2019 = pd.concat([rest_df, not_open_df, still_df], axis=0).reset_index(drop=True)
df_2019.sort_values(by=['인허가일자'], axis=0, inplace=True)

In [138]:
open_df = df_2019[df_2019['폐업일자'] >= 20200101].reset_index(drop=True)
closed_df = df_2019[(df_2019['폐업일자'] == 0) | (df_2019['폐업일자'] < 20200101)].reset_index(drop=True)
open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20050610,폐업,20200723,0,0,서울특별시 동대문구 청량리동 689,서울특별시 동대문구 약령시로 124,경동요양병원,127.043743,37.582935,2019
1,20060519,폐업,20200828,0,0,서울특별시 강남구 대치동 889,서울특별시 강남구 선릉로 435,네모치과병원,127.049331,37.504219,2019
2,20070309,폐업,20200712,0,0,서울특별시 강동구 명일동 335-3,서울특별시 강동구 양재대로 1596,러스크강동병원,127.143790,37.548084,2019
3,20110211,폐업,20200314,0,0,서울특별시 양천구 신월동 549-1,서울특별시 양천구 신월로 150,서울인화요양병원,126.840480,37.517063,2019
4,20120516,폐업,20200531,0,0,서울특별시 강남구 신사동 597,서울특별시 강남구 논현로164길 14,이지앤성형외과병원,127.029202,37.523452,2019
5,20121108,폐업,20200501,0,0,서울특별시 양천구 신정동 1023-1,서울특별시 양천구 신월로36길 17,서울그린치과병원,126.858628,37.520196,2019
6,20170628,폐업,20200831,0,0,서울특별시 노원구 공릉동 323-1,서울특별시 노원구 동일로 1107,서울스페셜병원,127.071070,37.628521,2019
7,20181101,폐업,20200817,0,0,서울특별시 강동구 길동 419,서울특별시 강동구 진황도로 122,서울실버한방병원,127.139414,37.534549,2019


In [139]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업중'
open_df

,인허가일자,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,지번주소,도로명주소,사업장명,경도,위도,년
0,20050610,영업중,0,0,0,서울특별시 동대문구 청량리동 689,서울특별시 동대문구 약령시로 124,경동요양병원,127.043743,37.582935,2019
1,20060519,영업중,0,0,0,서울특별시 강남구 대치동 889,서울특별시 강남구 선릉로 435,네모치과병원,127.049331,37.504219,2019
2,20070309,영업중,0,0,0,서울특별시 강동구 명일동 335-3,서울특별시 강동구 양재대로 1596,러스크강동병원,127.143790,37.548084,2019
3,20110211,영업중,0,0,0,서울특별시 양천구 신월동 549-1,서울특별시 양천구 신월로 150,서울인화요양병원,126.840480,37.517063,2019
4,20120516,영업중,0,0,0,서울특별시 강남구 신사동 597,서울특별시 강남구 논현로164길 14,이지앤성형외과병원,127.029202,37.523452,2019
5,20121108,영업중,0,0,0,서울특별시 양천구 신정동 1023-1,서울특별시 양천구 신월로36길 17,서울그린치과병원,126.858628,37.520196,2019
6,20170628,영업중,0,0,0,서울특별시 노원구 공릉동 323-1,서울특별시 노원구 동일로 1107,서울스페셜병원,127.071070,37.628521,2019
7,20181101,영업중,0,0,0,서울특별시 강동구 길동 419,서울특별시 강동구 진황도로 122,서울실버한방병원,127.139414,37.534549,2019


In [140]:
fin_2019_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2019_df.sort_values(by=['인허가일자'], axis=0, inplace=True)

In [141]:
fin_df = pd.concat([fin_2017_df, fin_2018_df, fin_2019_df], axis=0).reset_index(drop=True)
fin_df = fin_df[['년', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '인허가일자', '사업장명', '지번주소', '도로명주소', '위도', '경도']]
fin_df

,년,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,인허가일자,사업장명,지번주소,도로명주소,위도,경도
0,2017,영업중,0,0,0,19601124,서울특별시서북병원,서울특별시 은평구 역촌동 산 31-1,서울특별시 은평구 갈현로7길 49-1,37.604297,126.903688
1,2017,영업중,0,0,0,19610515,가톨릭대학교 은평성모병원,서울특별시 은평구 진관동 55,서울특별시 은평구 진관3로 15-35,37.642175,126.918375
2,2017,영업중,0,0,0,19620201,국립정신건강센터,서울특별시 광진구 중곡동 30-1,서울특별시 광진구 용마산로 127,37.565004,127.085767
3,2017,영업중,0,0,0,19660107,삼육서울병원,서울특별시 동대문구 휘경동 283-1,서울특별시 동대문구 망우로 82,37.589726,127.062542
4,2017,영업중,0,0,0,19660107,청량리정신병원,서울특별시 동대문구 청량리동 46-3,서울특별시 동대문구 제기로 144,37.586070,127.049088
...,...,...,...,...,...,...,...,...,...,...,...
1579,2019,영업중,0,0,0,20191030,엘치과병원,서울특별시 동대문구 답십리동 94-1,서울특별시 동대문구 전농로 110,37.574573,127.057846
1580,2019,영업중,0,0,0,20191031,미림한방병원,서울특별시 강남구 논현동 2,서울특별시 강남구 도산대로 110,37.516176,127.021058
1581,2019,영업중,0,0,0,20191213,서울스마트요양병원,서울특별시 양천구 신정동 1268,서울특별시 양천구 중앙로 181,37.512742,126.855774
1582,2019,영업중,0,0,0,20191213,연세바로척병원,서울특별시 성동구 성수동2가 279,서울특별시 성동구 성수동2가 아차산로13길 38,37.546292,127.062173


In [142]:
fin_df.to_csv('data/서울특별시 병원 인허가 정보 2017-2019(전처리 완료, cp949).csv', encoding='cp949', index=False)

In [144]:
df_2020 = new_addr_df[(new_addr_df['폐업일자']==0) | (new_addr_df['폐업일자']>20200101)].reset_index(drop=True)
df_2020 = df_2020[df_2020['인허가일자']<20210101]
df_2020['년'] = 2020
df_2020 = df_2020[['년', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '인허가일자', '사업장명', '지번주소', '도로명주소', '위도', '경도']]
df_2020

,년,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,인허가일자,사업장명,지번주소,도로명주소,위도,경도
0,2020,영업중,0,0,0,20190124,느루요양병원,서울특별시 강동구 성내동 452,서울특별시 강동구 강동대로 143-52,37.527865,127.120540
1,2020,영업중,0,0,0,19870424,세란병원,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,37.575170,126.957451
2,2020,영업중,0,0,0,20200708,보금한방병원,서울특별시 강서구 마곡동 731-11,서울특별시 강서구 방화대로34길 60,37.569973,126.819730
3,2020,영업중,0,0,0,19790324,강북삼성병원,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,37.568222,126.966701
4,2020,영업중,0,0,0,20151214,서울바른병원,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,37.500298,126.919996
...,...,...,...,...,...,...,...,...,...,...,...
540,2020,영업중,0,0,0,19840904,재단법인대성병원,서울특별시 서대문구 홍제동 234-2,서울특별시 서대문구 세무서길 20,37.588938,126.945792
541,2020,영업중,0,0,0,19740325,학교법인연세대학교의과대학세브란스병원,서울특별시 서대문구 신촌동 134,서울특별시 서대문구 연세로 50-1,37.563013,126.941721
542,2020,영업중,0,0,0,19710220,경찰병원,서울특별시 송파구 가락동 58,서울특별시 송파구 송이로 123,37.496415,127.123487
543,2020,휴업,0,20200801,20201231,20170106,서울동인병원,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,37.565111,127.028020


In [145]:
# 전출 row 삭제
idx = df_2020[df_2020['상세영업상태명']=='전출'].index
df_2020.drop(idx, axis=0, inplace=True)
df_2020

,년,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,인허가일자,사업장명,지번주소,도로명주소,위도,경도
0,2020,영업중,0,0,0,20190124,느루요양병원,서울특별시 강동구 성내동 452,서울특별시 강동구 강동대로 143-52,37.527865,127.120540
1,2020,영업중,0,0,0,19870424,세란병원,서울특별시 종로구 무악동 32-2,서울특별시 종로구 통일로16길 4-2,37.575170,126.957451
2,2020,영업중,0,0,0,20200708,보금한방병원,서울특별시 강서구 마곡동 731-11,서울특별시 강서구 방화대로34길 60,37.569973,126.819730
3,2020,영업중,0,0,0,19790324,강북삼성병원,서울특별시 종로구 평동 73,서울특별시 종로구 새문안로 29,37.568222,126.966701
4,2020,영업중,0,0,0,20151214,서울바른병원,서울특별시 영등포구 신길동 505,서울특별시 영등포구 여의대방로 145,37.500298,126.919996
...,...,...,...,...,...,...,...,...,...,...,...
540,2020,영업중,0,0,0,19840904,재단법인대성병원,서울특별시 서대문구 홍제동 234-2,서울특별시 서대문구 세무서길 20,37.588938,126.945792
541,2020,영업중,0,0,0,19740325,학교법인연세대학교의과대학세브란스병원,서울특별시 서대문구 신촌동 134,서울특별시 서대문구 연세로 50-1,37.563013,126.941721
542,2020,영업중,0,0,0,19710220,경찰병원,서울특별시 송파구 가락동 58,서울특별시 송파구 송이로 123,37.496415,127.123487
543,2020,휴업,0,20200801,20201231,20170106,서울동인병원,서울특별시 성동구 하왕십리동 1070-3,서울특별시 성동구 왕십리로 382,37.565111,127.028020


In [147]:
df_2020.to_csv('data/서울특별시 병원 인허가 정보 2020(전처리 완료, cp949).csv', encoding='cp949', index=False)